- Get inspiration for nearby lyrics
- Find history of a lyric

In [129]:
import requests
from re import sub
import lyricsgenius as genius

In [251]:
client_access_token = 'aKoyAcFSc5wKOQ7-_FWLU3sEPfT6H5av8vVNdiuP4lvEA1jWINFKr9nIiUaSXzVH'

In [242]:
def collect_lyrics(raw_lyric, auth_code, max_pages):
    '''
    Convert string to correct format, issue request
    and collect artist and song
    '''
    
    lyric_query = sub('\s+','%20', raw_lyric).lower()
    request_uri = 'https://api.genius.com/search/?q={}'.format(lyric_query)
    token = 'Bearer {}'.format(auth_code)
    headers = {'Authorization': token}
    
    current_page = 1
    next_page = True
    songs = []
    while next_page:
        params = {'page': current_page}
        print('Searching page {}....'.format(current_page))
        result = get_artists_and_songs(requests.get(request_uri, headers=headers, params=params))
        if result:
            songs += result
            current_page += 1
        else:
            next_page = False
        if current_page>max_pages:
            break
    return [song for song in songs if not raw_lyric.lower() in song[1].lower()]

def get_artists_and_songs(response):
    '''
    Convert request info to output of artist and song
    '''
    output = []
    for resp in response.json()['response']['hits']:
        try:
            output.append((resp['result']['title'],
                          resp['result']['primary_artist']['name'],
                          resp['result']['stats']['pageviews']))
        except:
            pass
        
    return output


In [254]:
output = collect_lyrics(input('What lyric would you like to look for?\n'), client_access_token, 10)

What lyric would you like to look for?
grey poupon
Searching page 1....
Searching page 2....
Searching page 3....
Searching page 4....
Searching page 5....
Searching page 6....
Searching page 7....
Searching page 8....
Searching page 9....
Searching page 10....


In [266]:
output[0][0:2]

('HUMBLE.', 'Kendrick Lamar', 10508314)

In [278]:
def prep_sdk(auth_code):
    '''
    Bring in genius sdk for more complex searches
    '''
    return genius.Genius(auth_code)

In [276]:
def extract_lyrics_for_song_artist(title, artist, genius_object):
    '''
    Collect all lyrics for a specific song
    '''
    song = api.search_song(title='HUMBLE.', artist='Kendrick Lamar')
    
    assert song.artist==artist and song.title==title
    return song.lyrics 

In [ ]:
def find_surrounding_lyrics(query, lyrics):
    '''
    Finds where a lyric is in context of the song
    '''
    

In [279]:
api = prep_sdk(client_access_token)

In [280]:
extract_lyrics_for_song_artist('HUMBLE.', 'Kendrick Lamar', api)

Searching for "HUMBLE." by Kendrick Lamar...
Done.


"[Intro]\nNobody pray for me\nIt's been that day for me\nWay (Yeah, yeah)\n\n[Verse 1]\nAyy, I remember syrup sandwiches and crime allowances\nFinesse a nigga with some counterfeits, but now I’m countin' this\nParmesan where my accountant lives, in fact I'm downin’ this\nD'USSÉ with my boo bae tastes like Kool-Aid for the analysts\nGirl, I can buy your ass the world with my paystub\nOoh, that pussy good, won't you sit it on my taste bloods?\nI get way too petty once you let me do the extras\nPull up on your block, then break it down: we playin' Tetris\nAM to the PM, PM to the AM, funk\nPiss out your per diem, you just gotta hate 'em, funk\nIf I quit your BM, I still ride Mercedes, funk\nIf I quit this season, I still be the greatest, funk\nMy left stroke just went viral\nRight stroke put lil' baby in a spiral\nSoprano C, we like to keep it on a high note\nIt's levels to it, you and I know\n\n[Chorus]\nBitch, be humble (Hol' up, bitch)\nSit down (Hol’ up, lil’, hol' up, lil’ bitch)\nBe 

In [267]:
temp = api.search_song(title='HUMBLE.', artist='Kendrick Lamar')

Searching for "HUMBLE." by Kendrick Lamar...
Done.


In [273]:
temp.lyrics, temp.artist, temp.title

("[Intro]\nNobody pray for me\nIt's been that day for me\nWay (Yeah, yeah)\n\n[Verse 1]\nAyy, I remember syrup sandwiches and crime allowances\nFinesse a nigga with some counterfeits, but now I’m countin' this\nParmesan where my accountant lives, in fact I'm downin’ this\nD'USSÉ with my boo bae tastes like Kool-Aid for the analysts\nGirl, I can buy your ass the world with my paystub\nOoh, that pussy good, won't you sit it on my taste bloods?\nI get way too petty once you let me do the extras\nPull up on your block, then break it down: we playin' Tetris\nAM to the PM, PM to the AM, funk\nPiss out your per diem, you just gotta hate 'em, funk\nIf I quit your BM, I still ride Mercedes, funk\nIf I quit this season, I still be the greatest, funk\nMy left stroke just went viral\nRight stroke put lil' baby in a spiral\nSoprano C, we like to keep it on a high note\nIt's levels to it, you and I know\n\n[Chorus]\nBitch, be humble (Hol' up, bitch)\nSit down (Hol’ up, lil’, hol' up, lil’ bitch)\nBe

In [256]:
def check_lyrics_in_context(auth_code, artist, song):
    '''
    Check the where the lyrics are found in
    their respective songs. Returns one sentence either side of the lyric.
    '''
    song_query = sub('\s+','%20', song).lower()
    request_uri = 'https://api.genius.com/songs/?text_format={}'.format(song_query)
    token = 'Bearer {}'.format(auth_code)
    headers = {'Authorization': token}